<a href="https://colab.research.google.com/github/HarryLatThinkCerca/ExploreData/blob/main/ThinkCERCA_Rubric_Predictions_2_harrychanges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install imbalanced-learn
!pip install scikit-plot
!pip install sentencepiece
!pip install lxml
from IPython.display import display, HTML
import os
import shutil
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
from scikitplot.metrics import plot_confusion_matrix, plot_roc
from sklearn.svm import SVR
from imblearn.under_sampling import RandomUnderSampler
import string
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel, BertConfig
from transformers.models.bert.modeling_bert import BertEncoder,BertPooler
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import BigBirdModel, BigBirdTokenizer, AutoTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
sns.set_style('darkgrid')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random.seed(42)

     |████████████████████████████████| 3.8 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 32.2 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 895 kB 34.7 MB/s 
     |████████████████████████████████| 6.5 MB 42.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
baseLoc = "/content/drive/Shareddrives/Engineering/Automated Assistance Team/aa_project_02_rsawa_data_collection/"
dataLoc = "aa_project_02_rubric_scores/flat_groupALL_rubric_8_scores_final_writing_8018.xlsx"

In [4]:
df = pd.read_excel(baseLoc+dataLoc)

In [6]:
group1 = df[df["grp_id"]==1]
group1 = group1[group1.apply(lambda x: len(str(x['sr_final_writing'])) > 100, axis=1)]

In [ ]:
group1

,id,grp_id,sa_id,itemresp_id,sa_assignment_id,a_rubric_id,a_grading_mode,l_id,ai_score1,ai_score2,ai_score3,ai_score4,ai_score5,ai_score6,ai_score7,ai_scoreX,sr_final_writing
0,1,1,12206123,32845,892611,252,rubric_score,26025,5,4,3,3,4,4,4,3.75,People get bullied multiple times every single...
1,2,1,12206124,32846,892611,252,rubric_score,26025,5,4,4,4,5,5,5,4.50,"After the 1999 Columbine shooting, which is kn..."
2,3,1,12206126,32847,892611,252,rubric_score,26025,3,3,4,4,4,4,5,3.80,Out of all things you can do bullying is one o...
3,4,1,12206127,32848,892611,252,rubric_score,26025,4,4,4,4,3,5,5,3.85,Bullying can lead to school shootings. Walter ...
4,5,1,12206128,32849,892611,252,rubric_score,26025,4,4,4,4,4,5,5,4.10,Bullying will not always lead to mass murder a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,2119,1,15603629,33793,1224125,252,rubric_score,26025,5,5,4,4,5,4,5,4.55,Have you wondered about the different opinions...
2007,2120,1,15603630,33776,1224125,252,rubric_score,26025,3,3,2,3,3,3,5,2.90,Walter Dean Myers point of view is that bullyi...
2008,2121,1,15603631,33775,1224125,252,rubric_score,26025,5,5,5,5,5,5,5,5.00,"1 out of every 5 students report bullying, and..."
2009,2122,1,15603633,33794,1224125,252,rubric_score,26025,1,1,1,1,1,1,1,1.00,"The Think cerca Editors did their research , w..."


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
xTrain, xTest, yTrain, yTest = train_test_split(group1["sr_final_writing"], group1["ai_scoreX"], test_size=0.2, random_state=42)
xTrain = xTrain.tolist()
xTest = xTest.tolist()
yTrain = yTrain.tolist()
yTest = yTest.tolist()
trainTokenized = [tokenizer(essay,padding="max_length",truncation=True,max_length=512,return_tensors='pt') for essay in xTrain]
trainData = TensorDataset(torch.stack([_["input_ids"] for _ in trainTokenized]).squeeze(),torch.stack([_["attention_mask"] for _ in trainTokenized]).squeeze(),torch.tensor(yTrain))
trainDL = DataLoader(trainData, batch_size=1)
testTokenized = [tokenizer(essay,padding="max_length",truncation=True,max_length=512,return_tensors='pt') for essay in xTest]
testData = TensorDataset(torch.stack([_["input_ids"] for _ in testTokenized]).squeeze(),torch.stack([_["attention_mask"] for _ in testTokenized]).squeeze(),torch.tensor(yTest))
testDL = DataLoader(testData, batch_size=1)

In [ ]:
class BigBirdForSequenceRegression(nn.Module):
    def __init__(self):
        super(BigBirdForSequenceRegression, self).__init__()
        self.bigbird = BertModel.from_pretrained("bert-base-uncased")
        self.linear1 = nn.Linear(768, 1)
        # self.linear2 = nn.Linear(256, 1)
    def forward(self, input_ids,attention_mask):
        #sequence_output, pooled_output = self.bigbird(input_ids=input_ids,attention_mask=attention_mask)
        sequence_output = self.bigbird(input_ids=input_ids,attention_mask=attention_mask)['last_hidden_state']
        linear1_output = self.linear1(sequence_output[:,0,:].view(-1,768))
        # linear2_output = self.linear2(linear1_output)
        return linear1_output
model = BigBirdForSequenceRegression().to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
yTrainRounded = [round(_) if _>0 and _<5.5 else (0 if _<0 else 5) for _ in yTrain]
yTestRounded = [round(_) if _>0 and _<5.5 else (0 if _<0 else 5) for _ in yTest]

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
epochs = 5
for epoch in range(1,epochs+1):
    print("=====Epoch %s=====" % epoch)
    model.train()
    train_loss = 0
    train_outputs = []
    for step,(input_ids,attention_mask,labels) in tqdm(enumerate(trainDL),position=0,leave=True):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids,attention_mask=attention_mask)
        loss = criterion(outputs,labels)
        train_loss += loss.item()
        train_outputs += outputs.detach().cpu().numpy()
        loss.backward()
        optimizer.step()
    print("Training Loss: %0.3f" % train_loss)
    train_labels = [round(_) if _>0 and _<5.5 else (0 if _<0 else 5) for _ in train_outputs]
    print(classification_report(y_true=yTrainRounded, y_pred=train_labels))
    plot_confusion_matrix(y_true=yTrainRounded,y_pred=train_labels)

=====Epoch 1=====


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
1389it [05:54,  3.92it/s]


KeyboardInterrupt: ignored